In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import sys
sys.path.append("/home/z5297792/UNSW-MRes/MRes/modules") 
from utils import dopioe, rossby_number, calc_tang_vel, find_directional_radii, psi_params, fit_psi_params

df_eddies = pd.read_pickle(f'/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Eddy_Data/df_eddies_processed_1462_{1766}.pkl')
df_eddies


,Eddy,Day,Cyc,Lon,Lat,ic,jc,xc,yc,w,q11,q12,q22,Rc,psi0,A,R,Age,Date,fname
0,1,1462,CE,156.969773,-28.461100,179,274,505.897600,1353.012251,-0.000033,1.099922,-0.193920,0.943344,104.792967,71.469206,-0.006508,69.500036,57,1994-01-02,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
1,1,1463,CE,156.993096,-28.433992,179,275,507.149717,1356.633687,-0.000030,1.066764,-0.119714,0.950849,102.305989,70.018058,-0.006690,52.250752,57,1994-01-03,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
2,1,1464,CE,157.097940,-28.381485,181,276,515.130000,1365.650728,-0.000028,1.016800,-0.154298,1.006892,115.245476,75.250400,-0.005666,67.000485,57,1994-01-04,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
3,1,1465,CE,157.094334,-28.315658,181,278,512.570772,1372.426342,-0.000033,0.896874,-0.276325,1.200119,90.907044,55.585304,-0.006726,52.001786,57,1994-01-05,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4,1,1466,CE,157.106324,-28.346479,181,277,514.728351,1369.599726,-0.000029,0.847358,-0.169788,1.214160,100.425496,58.705551,-0.005821,56.251116,57,1994-01-06,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4217,101,1762,AE,160.002623,-31.861445,255,222,893.309398,1095.131293,0.000018,0.814319,-0.492818,1.526269,47.881834,-10.121112,0.004415,3.765811,24,1994-10-29,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4218,101,1763,AE,160.072780,-31.787323,256,224,897.510405,1105.305100,0.000019,1.122813,-0.437232,1.275683,37.193545,-7.629876,0.004795,3.638420,24,1994-10-30,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4219,101,1764,AE,160.142938,-31.713201,257,226,901.711412,1115.478907,0.000019,1.431307,-0.381646,1.025098,26.505256,-5.138640,0.005176,3.511028,24,1994-10-31,/srv/scratch/z3533156/26year_BRAN2020/outer_av...
4220,101,1765,AE,160.213095,-31.639080,257,228,905.912420,1125.652714,0.000019,1.739801,-0.326060,0.774512,15.816967,-2.647403,0.005557,3.383636,24,1994-11-01,/srv/scratch/z3533156/26year_BRAN2020/outer_av...


In [2]:
import netCDF4 as nc
from scipy.interpolate import griddata
import netCDF4 as nc
from scipy.interpolate import RegularGridInterpolator
import time

# Field Data

fname = f'/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'

dataset = nc.Dataset(fname)

lon_rho = np.transpose(dataset.variables['lon_rho'], axes=(1, 0))
lat_rho = np.transpose(dataset.variables['lat_rho'], axes=(1, 0))
mask_rho = np.transpose(dataset.variables['mask_rho'], axes=(1, 0))
h =  np.transpose(dataset.variables['h'], axes=(1, 0))
angle = dataset.variables['angle'][0, 0]
z_r = np.load('/srv/scratch/z5297792/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))[150, 150, :]

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return EARTH_RADIUS * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

j_mid = lon_rho.shape[1] // 2
i_mid = lon_rho.shape[0] // 2

dx = distance(lat_rho[:-1, j_mid], lon_rho[:-1, j_mid],
              lat_rho[1:, j_mid], lon_rho[1:, j_mid])
dy = distance(lat_rho[i_mid, :-1], lon_rho[i_mid, :-1],
              lat_rho[i_mid, 1:], lon_rho[i_mid, 1:])

x_grid = np.insert(np.cumsum(dx), 0, 0)
y_grid = np.insert(np.cumsum(dy), 0, 0)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid, indexing='ij')


In [3]:
def compute_vert_data(df_eddies, eddy, X, Y, z_r, r=30):

    df = df_eddies[df_eddies.Eddy == eddy].copy()
    dic_data = {f'Eddy{eddy}': {}}
    fnumber_old = None
    x, y = X[:,0], Y[0,:]
    timer = 0

    # DOPIOE wont work if too close to boundary
    dx = np.max(np.diff(x))  # spacing in x-direction
    dy = np.max(np.diff(y))  # spacing in y-direction
    cell_size = np.max([dx, dy])        # max cell size in Euclidean units
    margin = int(np.ceil(r / cell_size))    

    for t, data in df.iterrows():
        day, fname, xc_surf, yc_surf, w_surf = data['Day'], data['fname'], data['xc'], data['yc'], data['w']
        fnumber = fname[-8:-3]

        # Only load if file changes
        if fnumber != fnumber_old:
            u_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/u_{fnumber}.npy')
            v_raw = np.load(f'/srv/scratch/z5297792/Climatology/u_v/v_{fnumber}.npy')
            u_raw = np.where(np.abs(u_raw) > 1e30, np.nan, u_raw).astype(float)
            v_raw = np.where(np.abs(v_raw) > 1e30, np.nan, v_raw).astype(float)
            with nc.Dataset(fname) as dataset:
                ocean_time = dataset.variables['ocean_time'][:].data / 86400
            fnumber_old = fnumber
        t_rel = np.where(ocean_time == day)[0][0]

        u_t, v_t = u_raw[:, :, :, t_rel], v_raw[:, :, :, t_rel]

        df_data = []
        xc, yc = xc_surf, yc_surf
        xc_old, yc_old = xc_surf, yc_surf

        # for k in range(24):
        for k in range(u_t.shape[-1]):
            u, v = u_t[:, :, k], v_t[:, :, k]

            R_grid = np.hypot(xc - X, yc - Y)
            ic, jc = map(int, np.unravel_index(np.argmin(R_grid), R_grid.shape))
            
            if (ic < margin or ic >= X.shape[0] - margin or
                jc < margin or jc >= X.shape[1] - margin):
                break

            # horizontal transect (constant y = y[jc])
            x_mask = np.abs(x - xc) <= r
            x1 = x[x_mask]
            y1 = np.full_like(x1, y[jc])
            u1 = u[x_mask, jc]
            v1 = v[x_mask, jc]
        
            # vertical transect (constant x = x[ic])
            y_mask = np.abs(y - yc) <= r
            y2 = y[y_mask]
            x2 = np.full_like(y2, x[ic])
            u2 = u[ic, y_mask]
            v2 = v[ic, y_mask]

            if np.any(np.isnan(u1)) or np.any(np.isnan(u2)):
                break

            xc, yc, w, Q, _, _, A0 = dopioe(x1, y1, u1, v1, x2, y2, u2, v2)

            if (np.sign(w) != np.sign(w_surf)) or (np.hypot(xc_old - xc, yc_old - yc) > 50):
                break
            
            xc_old, yc_old = xc, yc

            w *= 1e-3 # to s^-1

            radii = find_directional_radii(u, v, X, Y, xc, yc, calc_tang_vel)
            R = np.mean([radii['up'], radii['right'], radii['down'], radii['left']])

            q11, q12, q22 = Q[0,0], Q[0,1], Q[1,1]
            dx, dy = X - xc, Y - yc
            rho2 = q11*dx**2 + 2*q12*dx*dy + q22*dy**2
            rho_search = np.sqrt(np.where(rho2 < 0, np.nan, rho2))
            
            mask_outer = rho_search < min(R*1.75, 150)
            axi, ayi, aui, avi = X[mask_outer], Y[mask_outer], u[mask_outer], v[mask_outer]
            
            df = psi_params(xc, yc, Q, axi, ayi, aui, avi)
            Rc, psi0, A = fit_psi_params(df.rho2, df.Qr, df.vt, A0=A0, Rc_max=300)

            if np.sign(A) != np.sign(w):
                Rc, psi0, A = np.nan, np.nan, A0

            df_data.append({
                'x': xc, 'y': yc, 'Q': Q, 'w': w,
                'Rc': Rc, 'psi0': psi0, 'A': A, 'R': R,
                'z': k, 'Depth': z_r[k]
            })

        if df_data:
            df_data = pd.DataFrame(df_data)
            dx = df_data['x'].diff()
            dy = df_data['y'].diff()
            df_data['TD'] = np.hypot(df_data['x'] - df_data.iloc[0]['x'],
                                  df_data['y'] - df_data.iloc[0]['y'])
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = df_data
        else:
            dic_data[f'Eddy{eddy}'][f'Day{day}'] = pd.DataFrame(columns=['x', 'y', 'Q', 'w', 'Rc', 'psi0', 'A', 'R', 'z','Depth'])

        # if timer % 10 == 0:
        #     print(f'{timer / len(df) * 100:.1f}%')
        #     timer += 1
        
    return dic_data
    

In [4]:
dic_vert_eddies = {}
for e, eddy in enumerate(df_eddies.Eddy.unique()):
    # if eddy >= 2608:
    dic = compute_vert_data(df_eddies, eddy, X_grid, Y_grid, z_r)
    dic_vert_eddies = dic_vert_eddies | dic 

    with open(f"/srv/scratch/z5297792/Chapter2/SEACOFS_26yr_Eddy_Dataset/Eddy_Data/Vertical_Eddy_Dataset/dic_vert_eddies_{1}_{df_eddies.Eddy.max()}.pkl", "wb") as f:
        pickle.dump(dic_vert_eddies, f)

    if e % 10 == 0:
        print(f'{e+1}/{len(df_eddies.Eddy.unique())}')


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],


1/101


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be esti

11/101


/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, _ = curve_fit(vt_model, rho2, vt, p0=[A0, Rc0],
/home/z5297792/UNSW-MRes/MRes/modules/utils.py:450: OptimizeWarning: Covariance of the parameters could not be esti

KeyboardInterrupt: 